# OpenAI API 강의 7 — 멀티소스 RAG (YouTube · Web · News)


## 학습 목표
- 외부 데이터(YouTube, 웹, 뉴스)를 활용한 확장형 RAG 구조 이해
- LangChain과 OpenAI API를 통한 유튜브 자막 요약
- 실시간 웹 검색 + 뉴스 요약 결합 실습
- 멀티소스 검색 기반 Q&A/리포트 파이프라인 설계


## 0. 환경 준비 (필요시 실행)

In [1]:

# 필요시 다음을 실행하세요 (노트북/터미널 둘 다 가능)
# %pip install --upgrade langchain langchain-community langchain-openai openai tiktoken #     youtube-transcript-api feedparser tavily-python
# API Key 준비:
# - OpenAI:  export OPENAI_API_KEY=sk-...   (Windows: set OPENAI_API_KEY=...)
# - Tavily:  export TAVILY_API_KEY=tvly-...
print("필요 패키지: langchain, langchain-community, langchain-openai, openai, youtube-transcript-api, feedparser, tavily-python")

필요 패키지: langchain, langchain-community, langchain-openai, openai, youtube-transcript-api, feedparser, tavily-python


## 1. 공통 설정 및 안전한 임포트

In [1]:
import os

try:
    from langchain_openai import ChatOpenAI
except Exception:
    from langchain.chat_models import ChatOpenAI  # type: ignore

try:
    from langchain_community.tools.tavily_search import TavilySearchResults
except Exception:
    from langchain.tools import TavilySearchResults  # type: ignore

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
print("Imports OK")

Imports OK


## 2. 유튜브 영상 요약


| 항목 | 설명 |
|------|------|
| 목적 | 유튜브 자막을 불러와 핵심 내용 요약 |
| 주요 라이브러리 | `youtube_transcript_api`, `LangChain`, `OpenAI` |
| 응용 예시 | 강의 요약, 세미나 요약, 뉴스 인터뷰 자동 요약 |


In [1]:
# !pip install youtube_transcript_api

In [3]:
#  라이브러리 로드
import os
from dotenv import load_dotenv
from youtube_transcript_api import YouTubeTranscriptApi

from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnableSequence
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain.chains import RetrievalQA

#  환경 변수 로드
# load_dotenv()
# os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")


#  텍스트 분할 함수
def split_text(text):
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    return splitter.split_text(text)

#  요약 프롬프트
summary_prompt = PromptTemplate(
    input_variables=["text"],
    template="""
다음 텍스트 내용을 한국어로 요약해 주세요:

{text}
""")

#  LLM 및 요약 체인 정의
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
summary_chain = summary_prompt | llm | StrOutputParser()

### 유튜브 내용 요약하기
video_id = "4pYzYmSdSH4"  # 유튜브 링크의 v= 부분


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_47664\595874883.py:36: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)


In [9]:
## 유튜브 자막 추출하기
print("▶️ 유튜브 자막 추출 중... ", end='')

ytt_api = YouTubeTranscriptApi()
transcripts = ytt_api.fetch(video_id, languages=['ko', 'en'])
transcripts

▶️ 유튜브 자막 추출 중... 

FetchedTranscript(snippets=[FetchedTranscriptSnippet(text="I'm really excited for this next", start=5.52, duration=4.32), FetchedTranscriptSnippet(text="section. Uh, so we'll be doing a", start=8.08, duration=3.679), FetchedTranscriptSnippet(text='fireside chat with Andrew Wing. And', start=9.84, duration=3.679), FetchedTranscriptSnippet(text="Andrew probably doesn't need any", start=11.759, duration=3.441), FetchedTranscriptSnippet(text="introduction to most folks here. I'm", start=13.519, duration=3.281), FetchedTranscriptSnippet(text='guessing a lot of people have taken some', start=15.2, duration=4.239), FetchedTranscriptSnippet(text='of his classes on Corsera or deep', start=16.8, duration=5.04), FetchedTranscriptSnippet(text="learning. Um, but Andrew's also been a", start=19.439, duration=4.801), FetchedTranscriptSnippet(text='big part of the lane chain story. So, I', start=21.84, duration=4.72), FetchedTranscriptSnippet(text='met I met Andrew a little over two years', start=24.2

In [13]:
transcripts.snippets

[FetchedTranscriptSnippet(text="I'm really excited for this next", start=5.52, duration=4.32),
 FetchedTranscriptSnippet(text="section. Uh, so we'll be doing a", start=8.08, duration=3.679),
 FetchedTranscriptSnippet(text='fireside chat with Andrew Wing. And', start=9.84, duration=3.679),
 FetchedTranscriptSnippet(text="Andrew probably doesn't need any", start=11.759, duration=3.441),
 FetchedTranscriptSnippet(text="introduction to most folks here. I'm", start=13.519, duration=3.281),
 FetchedTranscriptSnippet(text='guessing a lot of people have taken some', start=15.2, duration=4.239),
 FetchedTranscriptSnippet(text='of his classes on Corsera or deep', start=16.8, duration=5.04),
 FetchedTranscriptSnippet(text="learning. Um, but Andrew's also been a", start=19.439, duration=4.801),
 FetchedTranscriptSnippet(text='big part of the lane chain story. So, I', start=21.84, duration=4.72),
 FetchedTranscriptSnippet(text='met I met Andrew a little over two years', start=24.24, duration=4.879)

In [16]:
transcript = " ".join([entry.text for entry in transcripts.snippets])
print("@ 유튜브 자막 추출 완료...!!! ")
print("\n📄 자막 추출 결과 (1000글자):\n", transcript[:1000])


@ 유튜브 자막 추출 완료...!!! 

📄 자막 추출 결과 (1000글자):
 I'm really excited for this next section. Uh, so we'll be doing a fireside chat with Andrew Wing. And Andrew probably doesn't need any introduction to most folks here. I'm guessing a lot of people have taken some of his classes on Corsera or deep learning. Um, but Andrew's also been a big part of the lane chain story. So, I met I met Andrew a little over two years ago at at at a conference um when we started talking about Lingchain and he he he graciously invited us to do a course on Lingchain with deep learning. I think it must have been the second or third one that they they they ever did and I know a lot of people here probably watched that course or got started on Lingchain because of that course. So, Andrew has been a huge part of the Lang Chain journey and I'm super excited to welcome him on stage for a fireside chat. So, let's welcome Andrew in. [Music] [Applause] Thanks for being [Music] here. But, by the way, Harrison was really kin

In [29]:
# pip install youtube-transcript-api --upgrade

from youtube_transcript_api import YouTubeTranscriptApi
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

video_id = "dQw4w9WgXcQ"  # 원하는 영상 ID

transcript = " ".join([entry.text for entry in transcripts.snippets])
print("@ 유튜브 자막 추출 완료...!!! ")

# LLM 기반 요약
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)
prompt = PromptTemplate(
    template="다음 유튜브 자막을 5문장으로 요약하고 마지막에 핵심 키워드 3개를 붙여라:\n{text}",
    input_variables=["text"]
)
chain = LLMChain(llm=llm, prompt=prompt)
yt_summary = chain.run(transcript)

print(yt_summary)


@ 유튜브 자막 추출 완료...!!! 
이번 세션에서는 Andrew Wing과의 대화가 진행되었으며, 그는 Langchain과 딥러닝 분야에서 중요한 역할을 해왔습니다. Andrew는 "에이전트"라는 개념에 대한 논의에서, 에이전트의 자율성 정도에 따라 시스템을 구분하는 것이 더 유용하다고 강조했습니다. 그는 비즈니스 프로세스에서 에이전틱 워크플로우를 구축하는 데 필요한 기술과 접근 방식에 대해 설명하며, 특히 평가 프레임워크의 중요성을 언급했습니다. 또한, 음성 기반 애플리케이션의 가능성과 AI 도구의 다양성을 강조하며, 이러한 도구들이 어떻게 더 나은 결과를 가져올 수 있는지를 설명했습니다. 마지막으로, 스타트업의 성공을 위한 핵심 요소로 속도와 기술적 지식을 강조했습니다.

핵심 키워드: 에이전트, Langchain, AI 도구


## 3. 웹 검색 결합 RAG (Web Search-Augmented QA)


| 항목 | 설명 |
|------|------|
| 목적 | 실시간 검색을 통해 최신 정보를 LLM 응답에 반영 |
| 도구 | `TavilySearchResults` (TAVILY_API_KEY 필요) |
| 응용 | 뉴스 요약, 최신 논문/트렌드 요약 |


In [28]:

# if not os.getenv("TAVILY_API_KEY"):
#     print("⚠️ TAVILY_API_KEY 환경변수가 필요합니다.")
# else:
#     search = TavilySearchResults(max_results=3)
#     llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)

#     query = "2025년 인공지능 산업 전망에 대한 최신 소식"
#     results = search.run(query)
#     results_text = str(results)

#     prompt = PromptTemplate(
#         template=(
#             "다음 검색 결과를 바탕으로 핵심 동향을 6문장으로 요약하고, "
#             "출처를 항목으로 나열하라. 마지막에 한 줄로 한글 TL;DR를 추가하라.\n\n"
#             "[검색 결과]\n{context}\n"
#         ),
#         input_variables=["context"]
#     )
#     chain = LLMChain(llm=llm, prompt=prompt)
#     web_summary = chain.run(results_text)
#     print(web_summary)

## 4. 뉴스 / 블로그 자동 요약 (RSS)


| 항목 | 설명 |
|------|------|
| 목적 | RSS/뉴스 API로 기사 수집 후 자동 요약 |
| 라이브러리 | `feedparser`, `LangChain` |
| 예시 | 기술 뉴스 브리핑 자동 생성 |


In [23]:

import feedparser

RSS_URL = "https://rss.nytimes.com/services/xml/rss/nyt/Technology.xml"
feed = feedparser.parse(RSS_URL)
type(feed)

feedparser.util.FeedParserDict

In [24]:
feed.entries

[{'title': 'A Mystery C.E.O. and Billions in Sales: Is China Buying Banned Nvidia Chips?',
  'title_detail': {'type': 'text/plain',
   'language': None,
   'base': 'https://rss.nytimes.com/services/xml/rss/nyt/Technology.xml',
   'value': 'A Mystery C.E.O. and Billions in Sales: Is China Buying Banned Nvidia Chips?'},
  'links': [{'rel': 'alternate',
    'type': 'text/html',
    'href': 'https://www.nytimes.com/2025/10/09/technology/nvidia-chips-china-megaspeed.html'},
   {'href': 'https://www.nytimes.com/2025/10/09/technology/nvidia-chips-china-megaspeed.html',
    'rel': 'standout',
    'type': 'text/html'}],
  'link': 'https://www.nytimes.com/2025/10/09/technology/nvidia-chips-china-megaspeed.html',
  'id': 'https://www.nytimes.com/2025/10/09/technology/nvidia-chips-china-megaspeed.html',
  'guidislink': False,
  'summary': 'An executive of a Singaporean firm called Megaspeed socialized with Nvidia’s Jensen Huang. Now the company is being scrutinized by U.S. officials for its ties t

In [25]:
items = []
for e in feed.entries[:3]:
    title = e.get("title", "")
    summary = e.get("summary", "")
    link = e.get("link", "")
    items.append(f"[{title}] {summary} (출처: {link})")

rss_text = "\n\n".join(items)
rss_text

'[A Mystery C.E.O. and Billions in Sales: Is China Buying Banned Nvidia Chips?] An executive of a Singaporean firm called Megaspeed socialized with Nvidia’s Jensen Huang. Now the company is being scrutinized by U.S. officials for its ties to China. (출처: https://www.nytimes.com/2025/10/09/technology/nvidia-chips-china-megaspeed.html)\n\n[Intel’s Big Bet: Inside the Chipmaker’s Make-or-Break Factory] The centerpiece of the company’s comeback hopes is a multibillion-dollar facility in the Arizona desert. Will it draw new customers? (출처: https://www.nytimes.com/2025/10/09/technology/intel-chip-factory-fab52.html)\n\n[What the Arrival of A.I. Video Generators Like Sora Means for Us] Welcome to the era of fakery. The widespread use of instant video generators like Sora will bring an end to visuals as proof. (출처: https://www.nytimes.com/2025/10/09/technology/personaltech/sora-ai-video-impact.html)'

In [26]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
prompt = PromptTemplate(
    template=(
        "다음 뉴스 3건을 항목별로 3문장 내외로 요약하고, 각각 한글 해시태그 2개를 붙여라.\n\n{news}"
    ),
    input_variables=["news"]
)
chain = LLMChain(llm=llm, prompt=prompt)
news_summary = chain.run(rss_text)
print(news_summary)

1. **[A Mystery C.E.O. and Billions in Sales: Is China Buying Banned Nvidia Chips?]**  
   싱가포르의 메가스피드라는 회사의 경영자가 엔비디아의 젠슨 황과 교류한 사실이 드러났다. 이로 인해 미국 당국은 해당 회사의 중국과의 연관성을 조사하고 있다. 이는 미국의 반도체 수출 규제와 관련된 중요한 사안으로 부각되고 있다.  
   **#엔비디아 #중국조사**

2. **[Intel’s Big Bet: Inside the Chipmaker’s Make-or-Break Factory]**  
   인텔은 애리조나 사막에 다수의 자금을 투자한 새로운 반도체 공장을 세우며 재도약을 노리고 있다. 이 공장이 새로운 고객을 유치할 수 있을지가 회사의 미래에 중요한 영향을 미칠 것으로 예상된다. 인텔의 성공 여부는 이 시설의 성과에 크게 달려 있다.  
   **#인텔 #반도체공장**

3. **[What the Arrival of A.I. Video Generators Like Sora Means for Us]**  
   AI 비디오 생성기인 소라의 출현으로 인해 시각적 증거의 신뢰성이 크게 저하될 것으로 보인다. 이러한 기술의 확산은 가짜 콘텐츠의 시대를 열고 있으며, 사회적 영향이 클 것으로 예상된다. 우리는 이제 시각적 정보의 진위를 더욱 신중하게 판단해야 할 필요성이 커졌다.  
   **#AI비디오 #가짜뉴스**


In [27]:
from IPython.display import Markdown, display
Markdown(news_summary)

1. **[A Mystery C.E.O. and Billions in Sales: Is China Buying Banned Nvidia Chips?]**  
   싱가포르의 메가스피드라는 회사의 경영자가 엔비디아의 젠슨 황과 교류한 사실이 드러났다. 이로 인해 미국 당국은 해당 회사의 중국과의 연관성을 조사하고 있다. 이는 미국의 반도체 수출 규제와 관련된 중요한 사안으로 부각되고 있다.  
   **#엔비디아 #중국조사**

2. **[Intel’s Big Bet: Inside the Chipmaker’s Make-or-Break Factory]**  
   인텔은 애리조나 사막에 다수의 자금을 투자한 새로운 반도체 공장을 세우며 재도약을 노리고 있다. 이 공장이 새로운 고객을 유치할 수 있을지가 회사의 미래에 중요한 영향을 미칠 것으로 예상된다. 인텔의 성공 여부는 이 시설의 성과에 크게 달려 있다.  
   **#인텔 #반도체공장**

3. **[What the Arrival of A.I. Video Generators Like Sora Means for Us]**  
   AI 비디오 생성기인 소라의 출현으로 인해 시각적 증거의 신뢰성이 크게 저하될 것으로 보인다. 이러한 기술의 확산은 가짜 콘텐츠의 시대를 열고 있으며, 사회적 영향이 클 것으로 예상된다. 우리는 이제 시각적 정보의 진위를 더욱 신중하게 판단해야 할 필요성이 커졌다.  
   **#AI비디오 #가짜뉴스**

## 5. 멀티소스 통합 리포트 생성


YouTube 요약, 웹 검색 요약, RSS 뉴스 요약을 하나의 컨텍스트로 통합하여
**'AI 산업 트렌드 요약 리포트'**를 생성합니다.


In [30]:

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)

context = (
    "### [YouTube 요약]\n" + (yt_summary if 'yt_summary' in globals() else "") + "\n\n" +
    # "### [웹 검색 요약]\n" + (web_summary if 'web_summary' in globals() else "검색 요약 없음") + "\n\n" +
    "### [뉴스 요약]\n" + (news_summary if 'news_summary' in globals() else "")
)

prompt = PromptTemplate(
    template=(
        "다음 통합 컨텍스트를 바탕으로 'AI 산업 트렌드 요약 리포트'를 작성하라.\n"
        "- 5개의 핵심 트렌드(굵은 글씨로 제목)와 각 3~4문장 설명\n"
        "- 마지막에 '실무 적용 팁' 섹션을 불릿으로 5개 작성\n\n"
        "{context}"
    ),
    input_variables=["context"]
)
chain = LLMChain(llm=llm, prompt=prompt)
final_report = chain.run(context)
print(final_report)

# AI 산업 트렌드 요약 리포트

## **1. 에이전트 기반 시스템의 발전**
에이전트라는 개념은 AI 시스템의 자율성 정도에 따라 구분될 수 있으며, 이는 비즈니스 프로세스의 효율성을 높이는 데 중요한 역할을 합니다. Andrew Wing은 에이전틱 워크플로우를 통해 어떻게 AI가 자율적으로 작업을 수행할 수 있는지를 설명하며, 이러한 시스템이 비즈니스에 미치는 긍정적인 영향을 강조했습니다. 특히, 평가 프레임워크의 중요성은 이러한 시스템의 성과를 측정하고 개선하는 데 필수적입니다.

## **2. AI 도구의 다양성과 활용**
AI 도구의 발전은 다양한 산업에서 혁신을 촉진하고 있습니다. Andrew는 음성 기반 애플리케이션과 같은 새로운 AI 도구들이 어떻게 더 나은 결과를 가져올 수 있는지를 설명하며, 이러한 도구들이 비즈니스의 경쟁력을 높이는 데 기여할 수 있음을 강조했습니다. 이러한 도구들은 사용자 경험을 개선하고, 데이터 분석 및 의사결정을 지원하는 데 중요한 역할을 합니다.

## **3. 반도체 산업의 재편**
인텔과 같은 반도체 제조업체들은 새로운 공장을 세우며 시장에서의 경쟁력을 강화하고 있습니다. 애리조나의 새로운 반도체 공장은 인텔의 미래에 중요한 영향을 미칠 것으로 예상되며, 이는 AI 기술의 발전과 밀접한 관련이 있습니다. 반도체 공급망의 안정성을 확보하는 것은 AI 기술의 발전과 상용화에 필수적입니다.

## **4. AI 비디오 생성기의 부상**
AI 비디오 생성기와 같은 새로운 기술의 출현은 시각적 콘텐츠의 신뢰성에 큰 도전을 안겨주고 있습니다. 이러한 기술은 가짜 콘텐츠의 확산을 촉진하며, 사회적 신뢰를 저하시킬 수 있는 잠재력을 가지고 있습니다. 따라서, 우리는 시각적 정보의 진위를 판단하는 데 더욱 신중해야 하며, AI 기술의 윤리적 사용에 대한 논의가 필요합니다.

## **5. 스타트업의 성공 요소**
스타트업의 성공은 속도와 기술적 지식에 크게 의존합니다. AI 기술을 활용하여 시장의 변화에 빠르게 대응하고, 혁신

In [31]:
Markdown(final_report)

# AI 산업 트렌드 요약 리포트

## **1. 에이전트 기반 시스템의 발전**
에이전트라는 개념은 AI 시스템의 자율성 정도에 따라 구분될 수 있으며, 이는 비즈니스 프로세스의 효율성을 높이는 데 중요한 역할을 합니다. Andrew Wing은 에이전틱 워크플로우를 통해 어떻게 AI가 자율적으로 작업을 수행할 수 있는지를 설명하며, 이러한 시스템이 비즈니스에 미치는 긍정적인 영향을 강조했습니다. 특히, 평가 프레임워크의 중요성은 이러한 시스템의 성과를 측정하고 개선하는 데 필수적입니다.

## **2. AI 도구의 다양성과 활용**
AI 도구의 발전은 다양한 산업에서 혁신을 촉진하고 있습니다. Andrew는 음성 기반 애플리케이션과 같은 새로운 AI 도구들이 어떻게 더 나은 결과를 가져올 수 있는지를 설명하며, 이러한 도구들이 비즈니스의 경쟁력을 높이는 데 기여할 수 있음을 강조했습니다. 이러한 도구들은 사용자 경험을 개선하고, 데이터 분석 및 의사결정을 지원하는 데 중요한 역할을 합니다.

## **3. 반도체 산업의 재편**
인텔과 같은 반도체 제조업체들은 새로운 공장을 세우며 시장에서의 경쟁력을 강화하고 있습니다. 애리조나의 새로운 반도체 공장은 인텔의 미래에 중요한 영향을 미칠 것으로 예상되며, 이는 AI 기술의 발전과 밀접한 관련이 있습니다. 반도체 공급망의 안정성을 확보하는 것은 AI 기술의 발전과 상용화에 필수적입니다.

## **4. AI 비디오 생성기의 부상**
AI 비디오 생성기와 같은 새로운 기술의 출현은 시각적 콘텐츠의 신뢰성에 큰 도전을 안겨주고 있습니다. 이러한 기술은 가짜 콘텐츠의 확산을 촉진하며, 사회적 신뢰를 저하시킬 수 있는 잠재력을 가지고 있습니다. 따라서, 우리는 시각적 정보의 진위를 판단하는 데 더욱 신중해야 하며, AI 기술의 윤리적 사용에 대한 논의가 필요합니다.

## **5. 스타트업의 성공 요소**
스타트업의 성공은 속도와 기술적 지식에 크게 의존합니다. AI 기술을 활용하여 시장의 변화에 빠르게 대응하고, 혁신적인 솔루션을 제공하는 것이 중요합니다. Andrew는 이러한 요소들이 스타트업의 경쟁력을 결정짓는 핵심 요소라고 강조하며, 기술적 전문성을 갖춘 팀의 중요성을 부각했습니다.

## 실무 적용 팁
- 에이전트 기반 시스템을 도입하여 비즈니스 프로세스를 자동화하고 효율성을 높이세요.
- 다양한 AI 도구를 활용하여 사용자 경험을 개선하고 데이터 분석을 강화하세요.
- 반도체 공급망의 안정성을 확보하기 위해 전략적 파트너십을 고려하세요.
- AI 비디오 생성기의 출현에 대비하여 콘텐츠의 진위를 검증하는 시스템을 마련하세요.
- 스타트업의 기술적 지식을 강화하기 위해 지속적인 교육과 훈련을 제공하세요.

## 6. 실습 과제


1) 관심 있는 주제의 유튜브 영상을 선택해 요약을 생성한다.  
2) Tavily 검색을 통해 최신 기사/블로그를 3~5건 수집해 요약한다.  
3) 다른 RSS 피드(예: 경제/과학)를 바꿔 요약 파이프라인을 실행한다.  
4) 세 소스를 통합하여 주간 리포트를 자동 생성한다.  
5) (선택) 결과를 Markdown 파일로 저장하고, 날짜별 폴더 구조를 만든다.


## 참고 자료


- LangChain: https://python.langchain.com/docs/  
- Tavily tool: https://python.langchain.com/docs/integrations/tools/tavily  
- YouTube Transcript API: https://pypi.org/project/youtube-transcript-api/  
- NYTimes RSS: https://developer.nytimes.com/docs/rss-api/1/overview
